# [빵이 궁금해]

## 1. 2020 빵소담 데이터 크롤링
#### -  지역별 크롤링
+ 서울 / 경기인천 / 대전충청 / 경상부산대구 / 강원 / 제주 

#### - 자동화해서 할 수 있는 전처리(R)
#### - 노가다 전처리
+ 빵집 이름 / 주소 추가
+ 추가 단어 처리 (소보로, 팥빵, 휘낭시에 등 빵단어 ㅋㅋㅋㅋ)
#### -  노가다로 채운 빵집리스트로 '빵집이름' 채우기

## 2. WordCloud
#### - 2020 빵소담, 어떤 단어가 가장 핫이슈인지: WordCloud

#### * idea
+ best 10 빵집 별 연관어 분석
+ 검색어 데이터 수집할 수 있나? 이것을 검색한 사람이 저것도 검색했다거나...

In [3]:
import pandas as pd
import time
import re
import csv
import numpy as np

# 명사 추출하기 위한 라이브러리 konlpy 설치
# 추출 명사 count를 위한 collection
from konlpy.tag import Okt
from collections import Counter

# 연관어 분석 패키지 로딩하기
import glob
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from gensim.models.word2vec import Word2Vec

# 네트워크 분석
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
import networkx as nx
import operator

# 시각화
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager
#import pytagcloud  # error : No module named 'pygame', 하지만 하고 싶은것은 돼서 그냥 두었다.
from matplotlib.pyplot import figure

### ---------------------------------------------- 노가다 ----------------------------------------------------------

### 1. 빵집 리스트 만들기 ><

In [7]:
def makeBreadList(bk):
    # 저장된 빵집이름 리스트화하기(빵집리스트 생성)
    # 1) df에서 빵집이름 컬럼 선택 
    bk_name = bk.loc[:,['빵집이름']]
    print("빵집이름 전체 갯수: "+str(len(bk_name)))

    # 2) NaN 데이터 제거
    bk_name1 = bk_name[bk_name['빵집이름'].notnull()]
    print("null 제거: "+str(len(bk_name1)))

    # 3) dataframe -> list 형변환
    bk_list = bk_name1.빵집이름.tolist()

    # 4) 한문자열에 여러 빵집쓰여있는 것 분리 후 저장, ("어글리 건담 오월의" -> "어글리", "건담", "오월의")
    dellist = []
    for i in range(0,len(bk_list)):
        split_str = bk_list[i].split()
        split_len = len(split_str)
        if split_len != 1:
            dellist.append(i)
            for j in range(0,split_len):
                bk_list.append(split_str[j])
        
    for i in range(0,len(dellist)):
        del bk_list[dellist[i]]

    # 5) 중복제거
    bk_list = list(set(bk_list))
    print("중복 제거: "+str(len(bk_list)))
    
    return bk_list

In [8]:
# 데이터 가져오기
bk = pd.read_csv('data-bakery_final_2020.csv')
makeBreadList(bk)

빵집이름 전체 갯수: 3970
null 제거: 3933
중복 제거: 1358


['‘파티세리소나',
 '엠마엔리아',
 '르솔레이 연호당',
 '서촌양과자점',
 '김용현베이커리',
 '마곡나루',
 '달쿠마',
 '송사부',
 '‘두빵집’,',
 '코코플랑',
 '러시아케이크',
 '파운야드베이크샵',
 '삼전-910',
 '플레이스앤',
 '어글리, 코코로카라',
 '하프파운드',
 '쿠펜브로트',
 '이누이',
 '통큰빵집',
 '업사이드커피',
 '본깔리떼',
 '루앙의부엉이부엌',
 '블랑제리',
 '혜민당',
 '미완성식탁',
 '에센브르',
 '쁘띠보뇌르',
 '파파도나스',
 '모던바이츠',
 '더벨로',
 '소락',
 '율교베이커리',
 '유동부',
 '위드브레드',
 '소울브레드 건강담은빵',
 '피터팬베이커리',
 '발롱디저트',
 '염창 블루밀',
 '플랑플랑',
 '양과',
 '다온브레드',
 '더브래드팬트리',
 '약',
 '소울브레드',
 '39도스콘',
 '쎄쎄종',
 '프레드릭',
 '효모빵집 드소영 디저티스트',
 '오보록 샤뽀블랑 디저트작업실',
 '가터알망',
 '로제또',
 '더소르아',
 '코른베르그',
 '꿈도미노910',
 '알프리베베이커리',
 '아쥬드블레 쉐프조',
 '브레드제이',
 '‘블랑제리코팡’,',
 '아이엠베이글 야쥬드블레',
 '약수역빵',
 '비밀 김용현베이커리 더브레드블루 더닐크팩토리 메종드꼼빠뇽',
 '약수역빵굼터',
 '쁘띠통',
 '파티세리소나',
 '건강담은빵 쟝블랑제리 소울브레드',
 '카페안데르센',
 '포비베이글',
 '본누벨',
 '안스베이커리',
 ' 아띠85도씨 풍년제과 더달달 힐튼제과 코른베르그 더브레드블루 다랑 약수역빵굼터 ',
 '카멜리온, 파티세리소나',
 '이름없는가게',
 '이제이베이킹',
 '골든치즈타르트',
 '브레드덕 렁트멍',
 '뺑드미 김병진과자점',
 '샤뽀블랑',
 '쁠로13 뺑오마담',
 '‘식빵이야기’',
 '마호가니',
 '이씨빵집',
 '밀곳같',
 '올드페리도넛',
 '루스티크 플라워베이커리',
 '카페오드',
 '아티장베이커

In [ ]:
bk_list = bk_name2.빵집이름.tolist()
bk_list.append('어글리')
bk_list.append('건담')
bk_list.append('오월의')
bk_list.append('이몸빵')
bk_list.append('솔브')
bk_list.append('장블')
bk_list.append('쟝블')
bk_list.append('장블랑제리')

In [26]:
# 인터넷에서 가져온 빵집 리스트
bklist_df = pd.read_csv('/Users/aohus/Documents/workspaces/Python/Studying/bklist.csv', encoding='utf-8') 
bklist = []
for col in bklist_df.columns: 
    bklist.append(col) 
    
for i in range (0, len(bklist)):
    bklist[i] = re.sub(r'\([^)]*\)', '', bklist[i])
#print(bk_list)

# 기존 리스트에 합치기
for i in range(0, len(bklist)):
    bk_list.append(bklist[i])

bk_list = list(set(bk_list))
print(bk_list)

['빵빵한식빵가게', '팡오르방', '쿠펜브로트', '폴인팬케이크', '위드르방', '갓브레드', '안국153', '디어브레드', '브레드숨', '베를린크란츠브로첸1552일하던분이오픈함 빵+까페 독일빵전문 식사빵 샌드위치)', '수마카롱', '키쉬미뇽', '금양식방', '홍철책빵', '미니말레minimale ', '우즈오븐', '엘리스파이', '무너미', '마므레', '대원당', '브레드몽드', '카페뎀셀브즈', '솔라스', '카페더스타', '우뺑', '벨르보', '쟝블', '더페이머스램', '피터팬베이커리', '브로아트', '우프', '미소빵긋', '업사이드커피', '마곡나루', '나우하우스', '효자베이커리', '메이비', '에뚜왈', '이누이카페', '율', '더베이커스테이블', '김진환제과점', '폴어반', '이웃집통통이', '랜디스도넛', '삼공(000)', '옵스', '그러팡야', '키쿠키', '카카베이커리', '힐튼제과', '어글리베이커리', '‘마리웨일마카롱’', '키커피컴퍼니', '카페블랑', '베이커스', '판데오로', '타르데마', '꿈꾸는크루아상', '설탕없는', '풍년제과', '루엘드파리,', '익투스', '마호가니', '라부아뜨', '호식이', '루프트발롱', '더벨로 ', '삼전-910', '리틀빅토리', '메종드아베크웰', '밀곳간', '케르반', '빵이좋은사람들', '베이크하우스', '해브온탑', '라두스', '소월길 밀영', '백록담', '야미요밀', '그레인킹', '카멜리온,', '헬로우베이글', '브리아몽', '이제이베이킹', '트루로멘스', '아모르버터', '꼬뺑', '동심베이커리', '블랑제리 가마', '바이레인', '츄이구이브레드(홍대 버터쫀득이치아바타 치아바타', '마이알레 ', '오갈미', '장블', '밤앤크루아상', '프레드릭베이커리', '소소하게', '고래빵집', '데이브레드2', '그레도제빵사', '에꼴드크로와상', '밤식빵', '미소한가득', '플라잉스콘', '익스큐즈미', '맘모스제과', '델리유', '황호

### 2. 빵데이터 제목에서 빵집이름 추출해서 저장

In [240]:
bk = pd.read_csv('/Users/aohus/Documents/workspaces/Python/Bread/data/data-bakery_cafe_2020_kw.csv', encoding='utf-8')

bk_title = bk.제목.tolist()

"""
a = bk_title[0].split()
print(a)
a[a.index('코코로카라')]
print(len(bk_title))
"""
pd.options.mode.chained_assignment = None   # 복제본 수정하면 원 데이터도 수정된다함....ㅎ 근데 귀차나서

for i in range(0, len(bk_title)):
    a =[]
    a = (set(bk_list) & set(bk_title[i].split()))
    if len(a) != 0 :
        a = str(a)
        bk.loc[i]['빵집이름'] = a

In [243]:
a = []
for i in range(0, len(bk_title)):
    b = list(set(bk_list) & set(bk_title[i].split()))
    a.append(b)
    
a = pd.Series(a)
bk['빵집이름'] = a.values
bk.to_csv('/Users/aohus/Documents/workspaces/Python/Studying/data-kwcafe.csv', sep=',', na_rep='NaN')
bk

,작성.날짜,제목,빵집이름,내용,닉넥임,댓글수,조회수
0,2020.08.02. 10:16,"\n 강릉 빵다방, 크림 처돌이 당장ㄱㄱ\n",[],※ 알림\n1. 본 게시판에서 100자미만의 게시글은 삭제대상이므로 반드시 100...,\n 고구마사줘\n,8,조회 729
1,2020.07.31. 11:03,\n 빵다방\n,[],녹차. 치즈. 먹물바케트도 구매했는사진은 흑임자만있네요 ㅋㅋㅋ저는 녹차가 제일 맛있...,\n 난나나나\n,10,조회 655
2,2020.07.29. 10:59,"\n 돌체테리아 초코나무숲, 찰떡시에, 솔티카라멜 휘낭시에\n ...",[],※ 알림1. 본 게시판에서 100자미만의 게시글은 삭제대상이므로 반드시 100자 이...,\n 쥬콩\n,3,조회 644
3,2020.07.26. 16:33,"\n 동내빵집(콘마요,빨미까레),대원당(버터크림빵,생크림 슈)\...",[],"매장명: 춘천 동내빵집.콘마요.한입 먹어보면 첫 맛인 윗부분은 고소한 콘마요, 우리...",\n Hannahh\n,2,조회 597
4,2020.07.22. 09:20,"\n 강릉 빵투어 3. 빵다방, 빵앗간\n",[],※ 알림1. 본 게시판에서 100자미만의 게시글은 삭제대상이므로 반드시 100자 이...,\n 일상을여행처럼\n,18,조회 967
...,...,...,...,...,...,...,...
218,2019.08.03. 11:43,\n (강릉) 돌체테리아\n,[],강릉으로 휴가다녀오면서 들른 곳이예요!강릉에 돌체테리아다녀왔구요!̤̻저는 ...,\n 챔프\n,2,"조회 1,265"
219,2019.08.02. 15:57,\n 강릉 베이커리가루\n,[],"가족휴가를 강릉으로 와서, 강릉엔 어떤 맛있는 빵집이 있나 하고 찾아보다가 빵소담에...",\n 작은나무7\n,11,"조회 1,558"
220,2019.07.29. 13:06,\n 돌체테리아 솔직 후기.. +바로방\n,[],※ 알림1. 본 게시판에서 100자미만의 게시글은 삭제대상이므로 반드시 100자 이...,\n 안녕하십니까아\n,14,"조회 1,893"
221,2019.07.24. 17:10,"\n 강릉 돌체테리아, 팡파미유 후기\n",[팡파미유],지난 비오는 주말에 강릉 돌체테리아와 팡파미유에 다녀왔습니다.돌체테리아는 오로지 초...,\n 헤이스팅스\n,5,"조회 7,267"


---------------------------------------------------------------------------------------------------------------

# 1. 빵소담 데이터 크롤링

## - 필요한 패키지

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import csv

## - 웹브라우저에 연결 및 로그인

In [192]:
driver = webdriver.Chrome('/Users/aohus/Documents/workspaces/Python/Studying/chromedriver')
driver.implicitly_wait(3)

In [193]:
# 로그인 전용 화면
driver.get('https://nid.naver.com/nidlogin.login?svctype=262144&url=http://m.naver.com/aside/')
# 아이디와 비밀번호 입력
driver.find_element_by_name('id').send_keys('dbtnghk528')
time.sleep(2)
driver.find_element_by_name('pw').send_keys('yoo004648!')
time.sleep(2)
# 로그인 버튼 클릭
# driver.find_element_by_css_selector('#frmNIDLogin > fieldset > input').click()

## - 카페 url 주소에 연결
   ### (1) 서울지역 카테고리의 글 주소('href') 크롤링
   ### (2) 각 게시물의 제목, 날짜, 글쓴이, 조회수, 댓글수 크롤링

In [205]:
base_url = "https://cafe.naver.com/thebbangs?iframe_url=/ArticleList.nhn%3Fsearch.clubid=24845809%26search.menuid=127%26search.boardtype=L%26search.totalCount=151%26search.page=1"
driver.get(base_url)

In [201]:
"""
# 구)먹은빵자랑 - [서울]
base_url = "https://cafe.naver.com/thebbangs?iframe_url=/ArticleList.nhn%3Fsearch.clubid=24845809%26search.menuid=110%26userDisplay=15%26search.headid=1%26search.boardtype=L%26search.totalCount=151%26search.page="
test_list = []
# for i in range(1,201):
    base_urli = base_url + str(i)
    
    driver.get(base_urli)
    # iframe으로 프레임 전환
    driver.switch_to.frame('cafe_main')

    article_list = driver.find_elements_by_css_selector('#main-area > div:nth-child(6) > table > tbody > tr > td.td_article > div.board-list > div > a.article')
    article_urls = [ i.get_attribute('href') for i in article_list ]

    for link in article_urls:
        test_list.append(link)

len(test_list)
"""
# 먹은빵자랑 - [전라]
base_url = "https://cafe.naver.com/thebbangs?iframe_url=/ArticleList.nhn%3Fsearch.clubid=24845809%26search.menuid=131%26userDisplay=50%26search.boardtype=L%26search.specialmenutype=%26search.totalCount=134%26search.page="
test_list = []
for i in range(1,4):
    base_urli = base_url + str(i)
    
    driver.get(base_urli)
    # iframe으로 프레임 전환
    driver.switch_to.frame('cafe_main')

    article_list = driver.find_elements_by_css_selector('#main-area > div:nth-child(6) > table > tbody > tr > td.td_article > div.board-list > div > a.article')
    article_urls = [ i.get_attribute('href') for i in article_list ]

    for link in article_urls:
        test_list.append(link)

# 구)먹은빵자랑 - [전라]
base_url = "https://cafe.naver.com/thebbangs?iframe_url=/ArticleList.nhn%3Fsearch.clubid=24845809%26search.menuid=110%26userDisplay=50%26search.headid=5%26search.boardtype=L%26search.totalCount=501%26search.page="
for i in range(1,3):
    base_urli = base_url + str(i)
    
    driver.get(base_urli)
    # iframe으로 프레임 전환
    driver.switch_to.frame('cafe_main')

    article_list = driver.find_elements_by_css_selector('#main-area > div:nth-child(6) > table > tbody > tr > td.td_article > div.board-list > div > a.article')
    article_urls = [ i.get_attribute('href') for i in article_list ]

    for link in article_urls:
        test_list.append(link)
        
print(len(test_list))

# 먹은빵자랑 - [강원]
base_url = "https://cafe.naver.com/thebbangs?iframe_url=/ArticleList.nhn%3Fsearch.clubid=24845809%26search.menuid=129%26userDisplay=50%26search.boardtype=L%26search.specialmenutype=%26search.totalCount=123%26search.page="
for i in range(1,4):
    base_urli = base_url + str(i)
    
    driver.get(base_urli)
    # iframe으로 프레임 전환
    driver.switch_to.frame('cafe_main')

    article_list = driver.find_elements_by_css_selector('#main-area > div:nth-child(6) > table > tbody > tr > td.td_article > div.board-list > div > a.article')
    article_urls = [ i.get_attribute('href') for i in article_list ]

    for link in article_urls:
        test_list.append(link)

# (구)먹은빵자랑 - [강원]
base_url = "https://cafe.naver.com/thebbangs?iframe_url=/ArticleList.nhn%3Fsearch.clubid=24845809%26search.menuid=110%26userDisplay=50%26search.headid=3%26search.boardtype=L%26search.totalCount=376%26search.page="
for i in range(1,3):
    base_urli = base_url + str(i)
    
    driver.get(base_urli)
    # iframe으로 프레임 전환
    driver.switch_to.frame('cafe_main')

    article_list = driver.find_elements_by_css_selector('#main-area > div:nth-child(6) > table > tbody > tr > td.td_article > div.board-list > div > a.article')
    article_urls = [ i.get_attribute('href') for i in article_list ]

    for link in article_urls:
        test_list.append(link)
        
print(len(test_list))

# 먹은빵자랑 - [충청]
base_url = "https://cafe.naver.com/thebbangs?iframe_url=/ArticleList.nhn%3Fsearch.clubid=24845809%26search.menuid=130%26userDisplay=50%26search.boardtype=L%26search.specialmenutype=%26search.totalCount=160%26search.page="
for i in range(1,5):
    base_urli = base_url + str(i)
    
    driver.get(base_urli)
    # iframe으로 프레임 전환
    driver.switch_to.frame('cafe_main')

    article_list = driver.find_elements_by_css_selector('#main-area > div:nth-child(6) > table > tbody > tr > td.td_article > div.board-list > div > a.article')
    article_urls = [ i.get_attribute('href') for i in article_list ]

    for link in article_urls:
        test_list.append(link)
        
len(test_list)

# (구)먹은빵자랑 - [충청]
base_url = "https://cafe.naver.com/thebbangs?iframe_url=/ArticleList.nhn%3Fsearch.clubid=24845809%26search.menuid=110%26userDisplay=50%26search.headid=4%26search.boardtype=L%26search.totalCount=501%26search.page="
for i in range(1,4):
    base_urli = base_url + str(i)
    
    driver.get(base_urli)
    # iframe으로 프레임 전환
    driver.switch_to.frame('cafe_main')

    article_list = driver.find_elements_by_css_selector('#main-area > div:nth-child(6) > table > tbody > tr > td.td_article > div.board-list > div > a.article')
    article_urls = [ i.get_attribute('href') for i in article_list ]

    for link in article_urls:
        test_list.append(link)
        
print(len(test_list))

# 먹은빵자랑 - [제주]
base_url = "https://cafe.naver.com/thebbangs?iframe_url=/ArticleList.nhn%3Fsearch.clubid=24845809%26search.menuid=133%26userDisplay=50%26search.boardtype=L%26search.specialmenutype=%26search.totalCount=101%26search.page="
for i in range(1,4):
    base_urli = base_url + str(i)
    
    driver.get(base_urli)
    # iframe으로 프레임 전환
    driver.switch_to.frame('cafe_main')

    article_list = driver.find_elements_by_css_selector('#main-area > div:nth-child(6) > table > tbody > tr > td.td_article > div.board-list > div > a.article')
    article_urls = [ i.get_attribute('href') for i in article_list ]

    for link in article_urls:
        test_list.append(link)
        

# (구)먹은빵자랑 - [제주]
base_url = "https://cafe.naver.com/thebbangs?iframe_url=/ArticleList.nhn%3Fsearch.clubid=24845809%26search.menuid=110%26userDisplay=50%26search.headid=7%26search.boardtype=L%26search.totalCount=359%26search.page="
for i in range(1,2):
    base_urli = base_url + str(i)
    
    driver.get(base_urli)
    # iframe으로 프레임 전환
    driver.switch_to.frame('cafe_main')

    article_list = driver.find_elements_by_css_selector('#main-area > div:nth-child(6) > table > tbody > tr > td.td_article > div.board-list > div > a.article')
    article_urls = [ i.get_attribute('href') for i in article_list ]

    for link in article_urls:
        test_list.append(link)
        
print(len(test_list))

234
457
767
918


In [202]:
len(test_list)

918

In [212]:
title_list1 = []
date_list1 = []
click_list1 = []
content_list1 = []
comment_list1 = []
name_list1 = []
#img_list = []

for i in range(767,918):
    driver.get(test_list[i])
    driver.switch_to.frame('cafe_main')
    time.sleep(2)
    html = BeautifulSoup(driver.page_source, 'html.parser')
    
    title = html.select('#app > div > div > div.ArticleContentBox > div.article_header > div.ArticleTitle > div > h3')
    date = html.select('#app > div > div > div.ArticleContentBox > div.article_header > div.WriterInfo > div > div.article_info > span.date')
    click = html.select('#app > div > div > div.ArticleContentBox > div.article_header > div.WriterInfo > div > div.article_info > span.count')
    content = html.select('#app > div > div > div.ArticleContentBox > div.article_container > div.article_viewer > div')
    comment = html.select('#app > div > div > div.ArticleContentBox > div.article_header > div.ArticleTool > a.button_comment > strong')
    name = html.select('#app > div > div > div.ArticleContentBox > div.article_header > div.WriterInfo > div > div.profile_info > div')
    #img = html.select('')
    
    title_text = title[0].text
    date_text = date[0].text
    click_text = click[0].text
    content_text = content[0].text
    comment_text = comment[0].text
    name_text = name[0].text
    #img_text = img[0].text
    
    title_list1.append(title_text)
    date_list1.append(date_text)
    click_list1.append(click_text)
    content_list1.append(content_text)
    comment_list1.append(comment_text)
    name_list1.append(name_text)
    #img_list.append(img_text)

In [213]:
print(len(test_list))
print(len(title_list1))
print(len(date_list1))
print(len(click_list1))
print(len(content_list1))

918
151
151
151
151


## - 크롤링한 텍스트 전처리
### (0) R에서 해야하나? csv로 저장하기 ** R로 했기 때문 ㅎㅎ
### (1) 필요없는 특수문자 제거
### (2) 명사 추출

In [214]:
# csv로 일단 저장

f = open('data-bakery_cafe_2020_jj.csv', 'w', encoding='UTF-8', newline='')
wr = csv.writer(f)

wr.writerow(['작성.날짜', '제목','빵집이름','내용','닉넥임','댓글수','조회수'])

for i in range(len(title_list1)):
    wr.writerow([date_list1[i], title_list1[i],'',content_list1[i],name_list1[i],comment_list1[i], click_list1[i]])

f.close()

In [7]:
# 명사 추출하기 위한 라이브러리 konlpy 설치
# 추출 명사 count를 위한 collection
from konlpy.tag import Okt
from collections import Counter

In [10]:
# 파일 불러오기
total = pd.read_csv('bakery_cafe_6yearsago.csv', encoding='utf-8')

In [11]:
content = total.loc[:,"내용"]
content.head(3)
#print(okt.morphs(total[1][3]))
#print(okt.(total[1][3]))

0       후기 매장 명 코코로카라 바나나오레오 카라벨라다크푸딩에 이어 바나나 미쯔말차 ...
1       후기 매장 명 우부래도일단 사진    죄송해요 제가       형이다보니 사람...
2       후기 매장 명 인라이크친구가 자기동네에 밀가루 안들어간 케이크 파는곳 생겼다고...
Name: 내용, dtype: object

In [12]:
# okt 객체 생성
okt = Okt()
word = []

# okt 단어 사전에 추가해주기


# 명사 추출
noun = []
for i in range(0,len(content)):
    noun += okt.nouns(content[i])

In [19]:
##################################
### 그냥 실험 morphs, pos 이용  #####
##################################

# 품사 포함 단어 추출
for i in range(0, len(content)):
    word += okt.morphs(content[i])
    
# 품사별 추출
pos = []
for i in range(0,len(content)):
    pos += okt.pos(content[i])

#print(word[0])
#print(noun[0])
#type(pos) : list
for i in range(20):
    if pos[i][1] != 'Josa':
        print(pos[i])

In [6]:
# 명사빈도 카운트
count = Counter(noun)
noun_list = count.most_common(150)

print(noun_list)

NameError: name 'Counter' is not defined

### (3) 불용어 처리

In [30]:
# 라이브러리 설치, 우와.... 터미널로 안가고 여기서 바로 할 수 있음
! pip3 install nltk

     |████████████████████████████████| 1.4 MB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 300 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 690 kB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 2.8 MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434676 sha256=9a70b55dfe57ef9c56ae476d10fc27521b0de1195a458ee76a3050a45ef64236
  Stored in directory: /Users/aohus/Library/Caches/pip/wheels/ff/d5/7b/f1fb4e1e1603b2f01c2424dd60fbcc50c12ef918bafc44b155
  Created wheel for regex: filename=regex-2020.7.14-cp38-cp38-macosx_10_9_x86_64.whl size=286082 sha256=917cf2e2b7cae4f78ad53c12cb355e7211f8ed041ef8194b04716afb9f4d82b2
  Stored in directory: /Users/aohus/Library/Caches/pip/wheels/c1/d2/6f/d7db2c6251fdcb9371d7d966eaddf630ac4e375f24252de399
Successfully built nltk regex


# 2. WordCloud

## - Tokenize

In [13]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [9]:
# 스택오버 플로우에서 얻어온 에러 해결책! 한번 실행했으니 두자

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
# 불용어 직접 지정해주었다. - 실패. 너무 길다
stop = "것, 저, 제, 더, 안, 이, 좀, 정말, 때, 분, 정도, 개, 후기, 전, 집, 그냥, 또, 도, 원, 다음, 달, 요거, 입, 다른, 수, 속, 점, 나, 류, 못, 은, 번, 시, 걸, 만, 덕"
stop_list = stop.split(',')

In [14]:
# stopword 함수 정의 - 실패. 너무 길다
def stopword(word_tokenize):
    result = []
    
    for w in word_tokenize:
        if w not in stop_list:
            result.append(w)
    return result

In [23]:
len(noun)

358607

In [20]:
# 불용어 제거 적용 - 실패. 너무 길다. 
# noun_list = []
# for i in range(0,len(noun)):
#     noun_list.append(stopword(noun))

KeyboardInterrupt: 

In [32]:
# 불용어 제거 후 명사빈도 카운트
count = Counter(noun)
noun150 = count.most_common(300)
noun_count = []
stop = "곳, 글, 역시, 요즘, 아주, 드, 비, 반, 글리, 것, 저, 제, 더, 안, 이, 좀, 정말, 때, 분, 정도, 개, 후기, 전, 집, 그냥, 또, 도, 원, 다음, 달, 요거, 입, 다른, 수, 속, 점, 나, 류, 못, 은, 번, 시, 걸, 만, 덕"

#print(noun150[0][0])
#print(stop_list)

for i in range(0,300):
    if noun150[i][0] not in stop:
        noun_count.append(noun150[i])
        
print(noun_count)

[('빵', 12250), ('맛', 7531), ('크림', 4577), ('치즈', 3362), ('버터', 2772), ('스콘', 2590), ('초코', 1974), ('느낌', 1713), ('바게트', 1691), ('식빵', 1619), ('팥', 1613), ('사진', 1449), ('빵집', 1421), ('맘모스', 1388), ('치아바타', 1331), ('향', 1327), ('케이크', 1316), ('밤', 1289), ('가격', 1094), ('방문', 1083), ('무화과', 1076), ('달달', 1050), ('단호박', 1013), ('매장', 1012), ('종류', 983), ('추천', 974), ('샌드위치', 973), ('쑥', 936), ('바삭', 928), ('식감', 926), ('겉', 921), ('말차', 900), ('브레드', 897), ('베이커리', 891), ('처음', 882), ('구매', 850), ('흑임자', 847), ('보고', 845), ('위', 833), ('조금', 822), ('쿠키', 808), ('파운드', 789), ('한번', 767), ('별로', 767), ('시간', 761), ('약간', 749), ('크로와상', 734), ('크기', 733), ('바로', 722), ('얼그레이', 712), ('근처', 706), ('딸기', 703), ('주문', 696), ('살짝', 692), ('재료', 686), ('이건', 686), ('날', 685), ('꼭', 675), ('생크림', 668), ('카페', 657), ('개인', 637), ('타르트', 625), ('포장', 621), ('호두', 608), ('중', 606), ('고구마', 598), ('커피', 595), ('해', 585), ('올리브', 579), ('제일', 575), ('바질', 573), ('단팥빵', 547), ('다시', 543), ('디저트', 541), 

## - WordCloud 시각화

In [33]:
from wordcloud import WordCloud

wc = WordCloud(font_path = '/Library/Fonts/Arial Unicode.ttf',\
              background_color = "white", \
              width = 1000, \
              height = 1000, \
              max_words = 150, \
              max_font_size = 300)

In [34]:
# 불용어 처리 전
wc.generate_from_frequencies(dict(noun_count))
wc.to_file('wordcloud_bakery1.png')